In [4]:
!pip install -U pandasql

ERROR: Could not find a version that satisfies the requirement pandasql (from versions: none)
ERROR: No matching distribution found for pandasql


In [52]:
#importing libraries

import pandas as pd
import pandasql as ps
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import warnings
warnings.filterwarnings("ignore")

#creating connections

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

repair_sheet_key='1kqbRWOVOMazHlGTz1G7ZWQFl1L2wVMw7KdVIJZCG55c'  

In [48]:
# Dm name
sheet= client.open_by_key('11D8_6u4ywy3yNYyrnMonyti6eflqUtpMfsxgLMBXEzk')#Car no master list
ws= sheet.worksheet('Current')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
df=dfm.iloc[:,[0,1]]
# sheet= client.open_by_key(repair_sheet_key)#Copy of Repairs - EFG Mumbai
# ws= sheet.worksheet('DM names')
# ws.clear() 
# ws.update([df.columns.values.tolist()] + df.values.tolist(),value_input_option='USER_ENTERED')
# print("Dm name updated succesfully")

In [56]:
#In Progress Repair

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('In Progress Repair')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_ipr=dfm.copy()#dump in progress repair
in_progress_repair_df=ps.sqldf(" select * from dfm where [Completion Error] = 'Completed' and [Verified by Floor Manager -Yes or No]='Yes' ")
in_progress_repair_df.drop('Final Status', axis=1,inplace=True)
print(in_progress_repair_df.head(10))

#In Progress Servicing

sheet= client.open_by_key(repair_sheet_key)#In progress repair
ws= sheet.worksheet('In Progress Servicing')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_ips=dfm.copy()#dump in progress repair
in_progress_servicing_df=ps.sqldf(" select * from dfm where [Completion Error] = 'Completed' and [Checked by Floor Incharge Y/N]='YES' or [Checked by Floor Incharge Y/N]='yes' ")
in_progress_servicing_df.drop('Completed Error', axis=1,inplace=True)
print(in_progress_servicing_df.head(10))

#Repair Labor

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Repair Labor')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_rl=dfm.copy()
repair_labor_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' ")
print(repair_labor_df.head(10))

#Servicing Labor

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Servicing Labor')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_sl=dfm.copy()
servicing_labor_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' ")
print(servicing_labor_df.head(10))

#Repair Parts

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Repair Parts')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
dump_rp=dfm.copy()
repair_parts_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' and [Quantity] >= 1 and [Price] >= 1")
dump_rp_completed_repair_parts=repair_parts_df.copy()
dump_rp_completed_repair_parts
repair_parts_df.drop(['P3 Current Quantity','Total Current Quantity','Ignore Erorr','Completed?','Total Quantity','Helper'], axis=1,inplace=True)
print(repair_parts_df.head(10))

#Servicing Parts

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Servicing Parts')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
dump_sp=dfm.copy()
servicing_parts_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' and [Quantity] >= 1 and [Price] >= 1 ")
dump_sp_completed_repair_parts=servicing_parts_df.copy()
servicing_parts_df.drop(['P3 Current Quantity','Total Current Quantity','Ignore Erorr','Completed?','Total Quantity','Helper'], axis=1,inplace=True)
print(servicing_parts_df.head(10))

             Timestamp   Car Number    Job Id Name of Driver  \
0  18/06/2022 10:30:38  MH01DR 2318  ERM15044                  
1  23/06/2022 10:46:55  MH01DR 2470  ERM15267        Mushtaq   
2  23/06/2022 11:15:53  MH01CV 7567  ERM15271                  
3  23/06/2022 11:30:25  MH01CV 0621  ERM15274                  
4  24/06/2022 09:59:40  MH01DR 2011  ERM15316        Pradeep   
5  24/06/2022 10:08:25  MH01CV 7768  ERM15320   Mhomad shafi   
6  24/06/2022 10:15:21  MH01CV 5219  ERM15322        Mehfuz    
7  24/06/2022 11:15:38  MH01DR 2335  ERM15327                  
8  24/06/2022 12:04:20  MH01DR 2720  ERM15337                  
9  24/06/2022 12:16:39  MH01CV 9239  ERM15340        Kailash   

  Contact number of Driver      Km       DM Name Name of Auditor  \
0                            82085     Red Eagle   Omkar Koyande   
1               7738227871   69781  Deadly Shark   Omkar Koyande   
2                           185943  Deadly Shark   Omkar Koyande   
3                      

     ERM ID                 Date   Car Number                Labor Code  \
0  ERM15044  25/06/2022 10:59:48  MH01DR 2318                Bumper R&R   
1  ERM15044  25/06/2022 10:59:48  MH01DR 2318  Panel Denting & painting   
2  ERM15044  25/06/2022 10:59:48  MH01DR 2318             Panel Denting   
3  ERM15044  25/06/2022 10:59:48  MH01DR 2318             Headlight R&R   
4  ERM15380  25/06/2022 11:46:58  MH01CV 7019             Brake service   
5  ERM15380  25/06/2022 11:46:58  MH01CV 7019     Blower motor Cleaning   
6  ERM15380  25/06/2022 11:46:58  MH01CV 7019             Condensor R&R   
7  ERM15366  25/06/2022 13:10:58  MH01CV 0617                Spark plug   
8  ERM15366  25/06/2022 13:10:58  MH01CV 0617                  Horn R&R   
9  ERM15366  25/06/2022 13:11:10  MH01CV 0617                  Bulb R&R   

  Technician Name Technician 2 Amount           Category Completed?  \
0          Vendor                 200          Body work  Completed   
1          Vendor               

In [ ]:
#In Progress Repair

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('In Progress Repair')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_ipr=dfm.copy()#dump in progress repair
in_progress_repair_df=ps.sqldf(" select * from dfm where [Completion Error] = 'Completed' and [Verified by Floor Manager -Yes or No]='Yes' ")
in_progress_repair_df.drop('Final Status', axis=1,inplace=True)
print(in_progress_repair_df.head(10))

In [3]:
#importing libraries

import pandas as pd
import pandasql as ps
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import warnings
warnings.filterwarnings("ignore")

#creating connections

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

repair_sheet_key='1kqbRWOVOMazHlGTz1G7ZWQFl1L2wVMw7KdVIJZCG55c'  

ModuleNotFoundError: No module named 'pandasql'

In [6]:
# #Repair Labor

# sheet= client.open_by_key(repair_sheet_key)
# ws= sheet.worksheet('Repair Labor')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# dump_rl=dfm.copy()
# repair_labor_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' ")
# repair_labor_df

,ERM ID,Date,Car Number,Labor Code,Technician Name,Technician 2,Amount,Category,Completed?,Duplicate Labor Entry


In [46]:
#Repair Parts

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Repair Parts')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
dump_rp=dfm.copy()
repair_parts_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' and [Quantity] >= 1 and [Price] >= 1")
dump_rp_completed_repair_parts=repair_parts_df.copy()
dump_rp_completed_repair_parts
repair_parts_df.drop(['P3 Current Quantity','Total Current Quantity','Ignore Erorr','Completed?','Total Quantity','Helper'], axis=1,inplace=True)
repair_parts_df

,JOB ID,Date,Car Number,Vehicle Brand,Part Number,Part Name,Vehicle Model,Brand Name,Correct Part?,Quantity,Price,Amount


In [9]:
#In Progress Servicing

sheet= client.open_by_key(repair_sheet_key)#In progress repair
ws= sheet.worksheet('In Progress Servicing')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
in_progress_servicing_df=ps.sqldf(" select * from dfm where [Completion Error] = 'Completed' and [Checked by Floor Incharge Y/N]='YES' or [Checked by Floor Incharge Y/N]='yes' ")
in_progress_servicing_df

,Timestamp,Car Number,JOB ID,Name of Driver,Contact number of Driver,Km,DM Name,Name of Auditor,Driver Remarks,In Time,...,Quantity of Spares,Actual Date of Handover,Handover to or Final Inspection by,Work Status,Completion Error,Remark,Start time,Out Time,Checked by Floor Incharge Y/N,Completed Error


In [10]:
#Servicing Labor

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Servicing Labor')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dump_sl=dfm.copy()
servicing_labor_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' ")

,JOB ID,Date,Car Number,Labor Code,Technician Name,Technician 2,Amount,Category,Completed?


In [47]:
#Servicing Parts

sheet= client.open_by_key(repair_sheet_key)
ws= sheet.worksheet('Servicing Parts')
data = ws.get_all_values()
headers = data.pop(0)
df = pd.DataFrame(data,columns=headers)
dfm=df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
dump_sp=dfm.copy()
servicing_parts_df=ps.sqldf(" select * from dfm where [Completed?] = 'Completed' and [Quantity] >= 1 and [Price] >= 1 ")
dump_sp_completed_repair_parts=servicing_parts_df.copy()
servicing_parts_df.drop(['P3 Current Quantity','Total Current Quantity','Ignore Erorr','Completed?','Total Quantity','Helper'], axis=1,inplace=True)

,JOB ID,Date,Car Number,Vehicle Brand,Part Number,Part Name,Vehicle Model,Brand Name,Correct Part?,Quantity,Price,Amount


In [40]:
df

,JOB ID,Date,Car Number,Vehicle Brand,Part Number,Part Name,Vehicle Model,Brand Name,Correct Part?,Quantity,Price,Amount,P3 Current Quantity,Total Current Quantity,Ignore Erorr,Completed?,Total Quantity,Helper,,
0,ESM12308,25/6/2022,MH01DR 5671,,Mobil Special 20W-50,Engine Oil,Celerio/Wagon R,Mobil,Yes,2.9,184.93,536.297,727.2,1351.2,Yes,No,1371.5,727.2,MH01DR 5671Mobil Special 20W-50,1
1,ESM12308,25/6/2022,MH01DR 5671,,16510M65L10,Oil Filter,Celerio/Wagon R,MGP,Yes,1,81.74,81.74,129,129,Yes,No,136,129,MH01DR 567116510M65L10,1
2,ESM12308,25/6/2022,MH01DR 5671,,F002H600208F8,Brake Fluid Dot 4,"Celerio, Wagon R & S Presso",Bosch,Yes,0.02,264.76,5.2952,11.97,11.97,Yes,No,12.36,11.97,MH01DR 5671F002H600208F8,1
3,ESM12308,25/6/2022,MH01DR 5671,,Coolant Mix (1:2),Coolant Mix (1:2),Celerio/Wagon R,Workshop,Yes,0.2,35.7,7.14,88,88,Yes,No,114.85,88,MH01DR 5671Coolant Mix (1:2),1
4,ESM12308,25/6/2022,MH01DR 5671,,DMDW01,Distilled Water Ltr,"Wagon R,Celerio",DM WATER,Yes,0.04,10.88,0.4352,116.8,116.8,Yes,No,117.44,116.8,MH01DR 5671DMDW01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,,,,,,,,,,,,0,,,,,,,,0
645,,,,,,,,,,,,0,,,,,,,,0
646,,,,,,,,,,,,0,,,,,,,,0
647,,,,,,,,,,,,0,,,,,,,,0
